# 프로젝트: 멋진 작사가 만들기

In [1]:
#라이브러리 파일 준비
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:        
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free', 'They come to join us here', 'From sea to shining sea And they all have a dream', 'As people always will', 'To be safe and warm', 'In that shining city on the hill Some wanna slam the door', 'Instead of opening the gate', "Aw, let's turn this thing around"]


1. 데이터 정제


앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [2]:
#필터링
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [3]:
#  정제 데이터 구축--준비 끝
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> verse <end>',
 '<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> in that shining city on the hill some wanna slam the door <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>']

In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #단어장 크기 12000이상
        filters=' ',  
        oov_token="<unk>"  
  
    )
   
    tokenizer.fit_on_texts(corpus)  
    tensor_tmp = tokenizer.texts_to_sequences(corpus) 
    
    tensor=[]
    for i in tensor_tmp:
        if len(i) > 15:
            continue
        else:
            tensor.append(i)
            
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  520    3 ...    0    0    0]
 [   2   45   66 ...    0    0    0]
 [   2    9 3390 ...    0    0    0]
 ...
 [   2  561   21 ...    0    0    0]
 [   2  120   34 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f1f7a174ed0>


In [5]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Verse 1]
They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
#단어사전 구축
src_input = tensor[:, :-1]   #tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성, 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2 520   3   0   0   0   0   0   0   0   0   0   0   0]
[520   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [8]:
#텐서 객체 생성--데이터 전처리 끝!!
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))  
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset  

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

2. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [9]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]  

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape) 

Source Train: (124810, 14)
Target Train: (124810, 14)


3. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1256
hidden_size = 2000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


In [11]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.6503956e-04, -8.0106338e-04, -2.2795475e-04, ...,
         -4.0653371e-04,  2.6256189e-04, -2.0669219e-04],
        [-1.2845028e-04, -1.1860488e-03, -3.6882932e-04, ...,
         -1.0265789e-03,  1.1403528e-03,  8.7334984e-06],
        [-3.3646566e-04, -1.0830673e-03, -4.1869804e-04, ...,
         -1.5648289e-03,  1.7942192e-03, -6.0915254e-04],
        ...,
        [-3.4838081e-03,  2.3006569e-03,  2.6971448e-04, ...,
         -2.6644810e-04,  9.0722681e-04, -1.8178547e-03],
        [-4.1034501e-03,  2.9375067e-03,  6.0810975e-04, ...,
          9.6332617e-05,  5.7243352e-04, -1.9354866e-03],
        [-4.6590259e-03,  3.4941996e-03,  8.9064625e-04, ...,
          3.7504040e-04,  2.5579936e-04, -2.0063724e-03]],

       [[-2.6503956e-04, -8.0106338e-04, -2.2795475e-04, ...,
         -4.0653371e-04,  2.6256189e-04, -2.0669219e-04],
        [-7.9980027e-04, -1.8439391e-03, -4.9613335e-04, ...,
         -1.1603427e-03, 

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  15073256  
_________________________________________________________________
lstm (LSTM)                  multiple                  26056000  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  32008000  
_________________________________________________________________
dense (Dense)                multiple                  24014001  
Total params: 97,151,257
Trainable params: 97,151,257
Non-trainable params: 0
_________________________________________________________________


In [13]:
#모델 학습
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=7, validation_data=(enc_val, dec_val))

Epoch 1/7
3901/3901 [==============================] - 1107s 283ms/step - loss: 3.1695 - val_loss: 2.6096
Epoch 2/7
3901/3901 [==============================] - 1109s 284ms/step - loss: 2.3792 - val_loss: 2.3633
Epoch 3/7
3901/3901 [==============================] - 1107s 284ms/step - loss: 1.8900 - val_loss: 2.2375
Epoch 4/7
3901/3901 [==============================] - 1108s 284ms/step - loss: 1.5283 - val_loss: 2.1963
Epoch 5/7
3901/3901 [==============================] - 1110s 285ms/step - loss: 1.2954 - val_loss: 2.2097
Epoch 6/7
3901/3901 [==============================] - 1108s 284ms/step - loss: 1.1596 - val_loss: 2.2540
Epoch 7/7
3901/3901 [==============================] - 1109s 284ms/step - loss: 1.0868 - val_loss: 2.2982


In [22]:
#모델 평가하기
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:  
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0]    == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = "" 
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
#생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '